### Import import libraries

In [126]:
import pandas as pd
import numpy as np

### Merge data and output

In [127]:
full=pd.merge(pd.read_csv("family_data.csv"),pd.read_csv("sample_submission.csv"))

### Reusable Functions

In [175]:
def prefCost(people,choice=-1):
    consolation=[[0,0],[50,0],[50,9],[100,9],[200,9],[200,18],[300,18],[300,36],[400,36],[500,36],[500,36+398]]
    return (consolation[choice][0]+consolation[choice][1]*people*1.0) 

def accCost(full):
    segregated=(full.groupby(by=['assigned_day']).sum())[['n_people']]
    segregated.loc[101]=segregated.loc[100]
    segregated['Cost']=0.0
    segregated=segregated.reset_index('assigned_day')
    #if((segregated['n_people']<125).any()):
    #    return -1
    if((segregated['n_people']>300).any()):
        return -1
    accounting_cost=0
    for day in range(100):
        diff=abs(segregated['n_people'][day]-segregated['n_people'][day+1])
        expPart=0.5+(diff/50.0)
        leftPart=(segregated['n_people'][day]-125.0)/400.0
        rightPart=segregated['n_people'][day] ** expPart
        segregated['Cost'][day]=(leftPart*rightPart)
    return segregated['Cost'].sum()

### Initial Assignment

In [130]:
#Assign -1 to all in full
full['assigned_choice']=-1
full['assigned_day']=-1
full['pref_cost']=-1

#Make a table for day-wise grouping
day_table=pd.DataFrame({'Day': range(1,101),'Count':0})
day_table.set_index('Day')


for day in range(0,100):
    print("Working for day "+str(day+1))
    print("=========================")
    day_table.loc[day]['Count']=0
    for family in range(5000):
        #rint("Working on "+str(family))
        if(day_table.loc[day]['Count']>=208):
            print("Total for day "+str(day+1)+"="+str(day_table.loc[day]['Count']))
            break
        if(full['assigned_day'][family]>-1):
            continue
        full['assigned_day'][family]=day+1
        day_table['Count'][day]+=full['n_people'][family]
        print("Adding family_id "+str(family))
        for choice in range(0,11):
            if(choice==10):
                full['assigned_choice'][family]=choice
                break
            if(full['choice_'+str(choice)][family]==day):
                full['assigned_choice'][family]=choice
                break
        
        full['pref_cost'][family]=prefCost(full['n_people'][family],full['assigned_choice'][family])
                
                
        

Working for day 1
Adding family_id 0
Adding family_id 1
Adding family_id 2
Adding family_id 3
Adding family_id 4
Adding family_id 5
Adding family_id 6
Adding family_id 7
Adding family_id 8
Adding family_id 9
Adding family_id 10
Adding family_id 11
Adding family_id 12
Adding family_id 13
Adding family_id 14
Adding family_id 15
Adding family_id 16
Adding family_id 17
Adding family_id 18
Adding family_id 19
Adding family_id 20
Adding family_id 21
Adding family_id 22
Adding family_id 23
Adding family_id 24
Adding family_id 25
Adding family_id 26
Adding family_id 27
Adding family_id 28
Adding family_id 29
Adding family_id 30
Adding family_id 31
Adding family_id 32
Adding family_id 33
Adding family_id 34
Adding family_id 35
Adding family_id 36
Adding family_id 37
Adding family_id 38
Adding family_id 39
Adding family_id 40
Adding family_id 41
Adding family_id 42
Adding family_id 43
Adding family_id 44
Adding family_id 45
Adding family_id 46
Adding family_id 47
Adding family_id 48
Adding famil

### Verify Total Count should be correct in assignment

In [131]:
day_table['Count'].sum()

21003

In [132]:
full['n_people'].sum()

21003

### Check initial cost

In [135]:
print("Acc Cost="+str(accCost(full)))
print("Pref Cost="+str(full['pref_cost'].sum()))

Acc Cost=378.6761161291622
Pref Cost=10574644


C:\Users\singgai\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### Start Optimization by looping over choices now

In [138]:
full_bk=full.copy()

In [156]:
full=full_bk.copy()

In [157]:
full

,family_id,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people,assigned_day,assigned_choice,pref_cost
0,0,52,38,12,82,33,75,64,76,10,28,4,1,10,2236
1,1,26,4,82,5,11,47,38,6,66,61,4,1,10,2236
2,2,100,54,25,12,27,82,10,89,80,33,3,1,10,1802
3,3,2,95,1,96,32,6,40,31,9,59,2,1,10,1368
4,4,53,1,47,93,26,3,46,16,42,39,4,1,10,2236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4995,16,1,66,33,18,70,56,46,86,60,4,100,10,2236
4996,4996,88,66,20,17,26,54,81,91,59,48,2,100,10,1368
4997,4997,32,66,54,17,27,21,74,81,3,7,6,100,10,3104
4998,4998,67,92,4,17,53,77,1,12,26,70,5,100,10,2670


In [180]:
def optimize(full,actUpon):
    print("Input Cost="+str(accCost(full)+full['pref_cost'].sum()))
    for family in range(5000):
        full_test=full.copy()
        curr_acc_cost=accCost(full)
        curr_pref_cost=full['pref_cost'].sum()
        curr_cost=curr_acc_cost+curr_pref_cost
        curr_choice=full_test['assigned_choice'][family]
        if(curr_choice!=actUpon):
            print("Skipping family_id "+str(family))
            continue
    
        print("Family Id:"+str(family)+" Cost="+str(curr_cost)+"(Pref="+str(full['pref_cost'][family])+")"+" Day="+str(full['assigned_day'][family])+" Choice="+str(full['assigned_choice'][family]))
        optimized_choice=full_test['assigned_choice'][family]
        optimized_cost=curr_cost
        for choice in range(10):
            full_test['assigned_day'][family]=full_test['choice_'+str(choice)][family]
            full_test['assigned_choice'][family]=choice
            full_test['pref_cost'][family]=prefCost(full_test['n_people'][family],choice)
            if(accCost(full_test)==-1):
                print("Leaving Choice "+str(choice))
                continue
            calculated_cost=accCost(full_test)+full_test['pref_cost'].sum()
            print("Choice="+str(choice)+" Calculated Cost="+str(calculated_cost)+" (Pref="+str(full_test['pref_cost'][family])+")")
            if(calculated_cost<optimized_cost):
                optimized_cost=calculated_cost
                optimized_choice=choice
        if(curr_choice!=optimized_choice):
            full['assigned_day'][family]=full['choice_'+str(optimized_choice)][family]
            full['assigned_choice'][family]=optimized_choice
            full['pref_cost'][family]=prefCost(full['n_people'][family],full['assigned_choice'][family])
        reduction=curr_cost-optimized_cost
        print("NEW: Family Id:"+str(family)+" Cost="+str(optimized_cost)+"(Pref="+str(full['pref_cost'][family])+")"+" Day="+str(full['assigned_day'][family])+" Choice="+str(full['assigned_choice'][family])+" Reduction="+str(reduction))
        print("=========================================================================")
    print("Output Cost="+str(accCost(full)+full['pref_cost'].sum()))
    return full

In [169]:
for i in range(10):
    full=optimize(full)

C:\Users\singgai\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Input Cost=9957652.721524352
Output Cost=5007805.8559616655
Input Cost=5007805.8559616655
Output Cost=1996100.2238930226
Input Cost=1996100.2238930226
Output Cost=966343.2822798522
Input Cost=966343.2822798522
Output Cost=689009.9569856303
Input Cost=689009.9569856303
Output Cost=642792.3191781971
Input Cost=642792.3191781971
Output Cost=627052.213785214
Input Cost=627052.213785214
Output Cost=616898.4234366711
Input Cost=616898.4234366711
Output Cost=604461.1041861987
Input Cost=604461.1041861987
Output Cost=595852.4970605267
Input Cost=595852.4970605267
Output Cost=594943.4820119757


In [ ]:
full=optimize(full,10)

In [183]:
full[full['assigned_choice']==10]
segregated=full.groupby(by=['assigned_day']).sum()
full.to_csv("optimized_data.csv")
segregated.to_csv("day_wise.csv")